In [105]:
import advent
rules, data = advent.get_lines_doublenewline(19)

In [106]:
from typing import NamedTuple
class Item(NamedTuple):
    x: int
    m: int
    a: int
    s: int

    def sum(self):
        return self.x + self.m + self.a + self.s

def parse_item(line: str) -> Item:
    # line looks like {x=787,m=2655,a=1222,s=2876}
    p = [int(l.split('=')[1]) for l in line[1:-1].split(',')]
    return Item(x=p[0], m=p[1], a=p[2], s=p[3])

def parse_rule(rule: str) -> tuple[str, list[str]]:
    name, rest = rule.split('{')
    rest = rest[:-1].split(',')
    return name, rest


def apply_rule(item: Item, rules: list[str]) -> str:
    # Returns 'A', 'R', or a name like 'qkq'
    for rule in rules:
        if ':' not in rule: return rule
        cond, result = rule.split(':')
        l, c, r = cond[0], cond[1], int(cond[2:])
        if c == '>' and getattr(item, l) > r: return result
        elif c == '<' and getattr(item, l) < r: return result
    assert False, "None of the rules match"


assert parse_rule('px{a<2006:qkq,m>2090:A,rfg}') == ('px', ['a<2006:qkq', 'm>2090:A', 'rfg'])
assert apply_rule(Item(1, 2, 3, 4), ['a>2000:q', 's<2000:v', 'z']) == 'v'

In [107]:
# Solve part 1

def solve_item(item: Item, rules: dict[str, list[str]]):
    rule = apply_rule(item, rules['in'])
    while rule not in ['A', 'R']:
        rule = apply_rule(item, rules[rule])
    return rule

parsed_rules = dict(parse_rule(rule) for rule in rules)
parsed_items = [parse_item(i) for i in data]

result = 0
for item in parsed_items:
    if solve_item(item, parsed_rules) == 'A':
        result += item.sum()
print(result)

446517


In [108]:
class ItemRange(NamedTuple):
    # List of ranges of min, max. ranges are INCLUSIVE
    x: list[tuple[int, int]]
    m: list[tuple[int, int]]
    a: list[tuple[int, int]]
    s: list[tuple[int, int]]

    @staticmethod
    def all():
        return ItemRange([], [], [], [])

    def add(self: 'ItemRange', other: 'ItemRange'):
        # Takes the intersection
        return ItemRange(self.x + other.x, self.m + other.m, self.a + other.a, self.s + other.s)
    
    def size(self):
        # Calculates size of this itemrange by brute force
        result = 1
        for attr in ['x', 'm', 'a', 's']:
            ranges = getattr(self, attr)
            result *= sum(all(i >= r[0] and i <= r[1] for r in ranges) for i in range(1, 4001))
        return result
    
    @staticmethod
    def from_dict(d: dict[str, list[tuple[int, int]]]):
        result = ItemRange.all()
        return ItemRange(
            x = result.x + (d['x'] if 'x' in d else []),
            m = result.m + (d['m'] if 'm' in d else []),
            a = result.a + (d['a'] if 'a' in d else []),
            s = result.s + (d['s'] if 's' in d else [])
        )
    
foo = ItemRange([(1, 5)], [(1000, 2000)], [], []).add(ItemRange.all())
assert foo.size() == 5 * 1001 * 4000 * 4000

In [112]:
parsed_rules: dict[str, list[str]] = dict(parse_rule(rule) for rule in rules)
# Kinda hacky ad-hoc rules, but this is easier than refactoring my code :)
parsed_rules['A'] = ['A']
parsed_rules['R'] = ['R']

def find_all_accepting_ranges(rules: list[str], currange: ItemRange = ItemRange.all()) -> list[ItemRange]:
    global parsed_rules
    result: list[ItemRange] = []
    for rule in rules:
        # currange is the range you must meet to encounter this rule in the first place
        if rule == 'A':
            result += [currange]
            break
        if rule == 'R':
            break
        if ':' not in rule:
            result += find_all_accepting_ranges(parsed_rules[rule], currange)
            break

        cond, state = rule.split(':')
        l, c, r = cond[0], cond[1], int(cond[2:])

        if c == '<':
            condrange = ItemRange.from_dict({l: [(0, r-1)]})
            oppositerange = ItemRange.from_dict({l: [(r, 4000)]})
        else:
            condrange = ItemRange.from_dict({l: [(r+1, 4000)]})
            oppositerange = ItemRange.from_dict({l: [(0, r)]})
        
        # Two branches: either we take condrange, or we continue the loop with oppositerange
        result += find_all_accepting_ranges(parsed_rules[state], currange.add(condrange))
        currange = currange.add(oppositerange)
        
    return result

accepting = find_all_accepting_ranges(parsed_rules['in'])

sum(acc.size() for acc in accepting)
# find_all_accepting_ranges function is instant, the sum(acc.size()) takes ~3 seconds

130090458884662